In [ ]:
%pip install boto3 litellm aiohttp --quiet --upgrade

<div class="alert alert-block alert-info">
<center>⚠️ <b>Important:</b> Please restart the kernel after installing the dependencies. ⚠️</center>
</div>

# Understanding Function/Tool Calling

Modern AI agents excel at tool utilization—the ability to identify, select, and leverage appropriate tools from their available arsenal to accomplish specific tasks. These tools might include:

- Database queries
- Computational functions
- Document processing capabilities
- Integrations with specialized services

The agent orchestrates these tools in sequence, creating workflows that seamlessly transition between different capabilities as needed. This approach allows systems to transcend the limitations of any single model or tool.

Let's start with a sample tool - a very simple Python function:

In [ ]:
def get_top_song(sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise Exception(f"Station {sign} not found.")

    return {
        "song": song, 
        "artist": artist
    }

In order for the LLM to know that it can use this tool, we have to pass the tool definition to the LLM.

In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_top_song",
                "description": "Get the most popular song played on a radio station.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ and WKRP."
                            }
                        },
                        "required": [
                            "sign"
                        ]
                    }
                }
            }
        }
    ]
}

Now we can start conversing with the model.

In [ ]:
input_text = "What is the most popular song on WZPZ?"

In [ ]:
import boto3

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=boto3.Session().region_name)
messages = [{
        "role": "user",
        "content": [{"text": input_text}]
}]

response = bedrock_client.converse(
    modelId="us.amazon.nova-pro-v1:0",
    messages=messages,
    toolConfig=tool_config
)

In [ ]:
output_message = response['output']['message']
messages.append(output_message)

In [ ]:
response['output']['message']['content']

In [ ]:
for tool_request in response['output']['message']['content']:
    if 'toolUse' in tool_request:
        tool = tool_request['toolUse']
        tool_name = tool['name']
        tool_input = tool['input']
        print(tool)

In [ ]:
output = get_top_song(**tool_input)
tool_result = {
    "toolUseId": tool['toolUseId'],
    "content": [{"json": output}]
}
tool_result

In [ ]:
tool_result_message = {
    "role": "user", "content": [{"toolResult": tool_result}]
}
messages.append(tool_result_message)

In [ ]:
response = bedrock_client.converse(
    modelId="us.amazon.nova-pro-v1:0",
    messages=messages,
    toolConfig=tool_config
)
output_message = response['output']['message']
output_message

For your ease of use, here we provide a function that calls tools until a final response is achieved:

In [ ]:
import sys
from typing import Any, Dict, List, Tuple

def converse_with_tools(user_input: str, tools: Dict[str, Any], verbose: bool = False) -> Tuple[Dict[str, Any], Dict[str, int]]:
    """
    Converse with a model using a tool-assisted interaction loop and track token usage.

    Args:
        user_input: Initial user query.
        tools: Tool configuration dictionary for the model.
        verbose: If True, print intermediate steps.

    Returns:
        A tuple containing:
            - Final response from the model after tool usage (if any).
            - Dictionary with cumulative token usage.
    """
    
    def log(msg):
        if verbose:
            print(msg)

    def call_tool(tool_name: str, tool_input: Dict[str, Any]) -> Dict[str, Any]:
        """Call a local function as a tool by name with arguments."""
        try:
            return getattr(sys.modules[__name__], tool_name)(**tool_input)
        except Exception as e:
            raise RuntimeError(f"Error calling tool '{tool_name}': {e}")

    # Start the conversation
    messages = [{
        "role": "user",
        "content": [{"text": user_input}]
    }]

    # Initialize token usage tracking
    total_usage = {
        "inputTokens": 0,
        "outputTokens": 0,
        "totalTokens": 0,
    }

    def update_usage(response: Dict[str, Any]):
        usage = response.get("usage", {})
        for key in total_usage:
            total_usage[key] += usage.get(key, 0)

    # First model call
    response = bedrock_client.converse(
        modelId="us.amazon.nova-pro-v1:0",
        messages=messages,
        toolConfig=tools
    )
    update_usage(response)

    while True:
        output_message = response['output']['message']
        messages.append(output_message)

        content = output_message.get("content", [])
        if len(content) < 2 or "toolUse" not in content[1]:
            break  # No more tools to use

        tool_use = content[1]["toolUse"]
        tool_name = tool_use["name"]
        tool_input = tool_use["input"]
        tool_use_id = tool_use["toolUseId"]

        log(f"[Tool Call] {tool_name} with input {tool_input}")
        tool_output = call_tool(tool_name, tool_input)

        tool_result_message = {
            "role": "user",
            "content": [{
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [{"json": tool_output}]
                }
            }]
        }

        log(f"[Tool Result] {tool_result_message}")
        messages.append(tool_result_message)

        # Continue the conversation
        response = bedrock_client.converse(
            modelId="us.amazon.nova-pro-v1:0",
            messages=messages,
            toolConfig=tools
        )
        update_usage(response)

    return response['output']['message']['content'][0]['text'], total_usage


In [ ]:
# Example usage
user_input = "What is the most popular song on WZPZ?"
converse_with_tools(user_input, tool_config)

## Amazon Bedrock - Function Calling with LiteLLM

Amazon Bedrock uses a different tool definition from the OpenAI Completion standard. Therefore, we have to change the definition before passing it to LiteLLM.

In [ ]:
def bedrock_tools_to_litellm_tools(bedrock_tools: dict)-> List[Dict[str, Any]]:
    """
    Convert Bedrock tool configuration to a format compatible with Litellm.

    Args:
        bedrock_tools: Bedrock tool configuration dictionary.

    Returns:
        A list of dictionaries representing the tools in a format compatible with Litellm.
    """
    litellm_tools = []
    for tool in bedrock_tools['tools']:
        litellm_tool = {
            "type": "function",
            "function": {
                "name": tool['toolSpec']['name'],
                "description": tool['toolSpec']['description'],
                "parameters": {
                    "type": "object",
                    "properties": tool['toolSpec']['inputSchema']['json']['properties'],
                    "required": tool['toolSpec']['inputSchema']['json'].get('required', [])
                }
            }
        }
        litellm_tools.append(litellm_tool)
    return litellm_tools

def litellm_tools_to_bedrock_tools(litellm_tools: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Convert Litellm tool configuration to a format compatible with Bedrock.

    Args:
        litellm_tools: List of dictionaries representing the tools in a format compatible with Litellm.

    Returns:
        A dictionary representing the tools in a format compatible with Bedrock.
    """
    bedrock_tools = {
        "tools": []
    }
    for tool in litellm_tools:
        bedrock_tool = {
            "toolSpec": {
                "name": tool['function']['name'],
                "description": tool['function']['description'],
                "inputSchema": tool['function']['parameters']
            }
        }
        bedrock_tools['tools'].append(bedrock_tool)
    return bedrock_tools

In [ ]:
import litellm
import sys
import json


messages = [
    {'role':'system', 'content':"You are a helpful assistant."},
    {'role':'user', 'content':input_text}
]

response = litellm.completion(
    model="bedrock/us.amazon.nova-pro-v1:0",
    messages=messages,
    tools=bedrock_tools_to_litellm_tools(tool_config),
)
messages.append(response['choices'][0]['message'])
messages

With the complete function loop:

In [ ]:
import sys
import json
from typing import Dict, List, Tuple, Any
import litellm

def converse_with_tools_litellm(
    input_text: str,
    tool_config: Dict[str, Any],
    verbose: bool = False
) -> Tuple[Dict[str, Any], Dict[str, int]]:
    """
    Converse using LiteLLM with Bedrock and tool support. Tracks token usage.

    Args:
        input_text: User query.
        tool_config: Tool configuration for litellm.
        verbose: Whether to log intermediate steps.

    Returns:
        A tuple of (final assistant message, token usage dictionary).
    """
    def log(msg):
        if verbose:
            print(msg)

    def call_tool(name: str, args: str) -> Any:
        try:
            parsed_args = json.loads(args)
            return getattr(sys.modules[__name__], name)(**parsed_args)
        except Exception as e:
            raise RuntimeError(f"Error calling tool '{name}': {e}")

    def extract_tokens(response: Dict[str, Any]) -> Dict[str, int]:
        usage = response.get("usage", {})
        return {
            "inputTokens": usage.get("prompt_tokens", 0),
            "outputTokens": usage.get("completion_tokens", 0),
            "totalTokens": usage.get("total_tokens", 0)
        }

    messages: List[Dict[str, Any]] = [
        {'role': 'system', 'content': "You are a helpful assistant."},
        {'role': 'user', 'content': input_text}
    ]

    # Initial call
    response = litellm.completion(
        model="bedrock/us.amazon.nova-pro-v1:0",
        messages=messages,
        tools=tool_config,
    )
    usage_total = extract_tokens(response)
    message = response['choices'][0]['message']
    messages.append(message.__dict__)

    # Process tool calls if any
    if getattr(message, "tool_calls", None):
        for tool_call in message.tool_calls:
            log(f"[Tool] Calling: {tool_call.function.name} with args {tool_call.function.arguments}")
            result = call_tool(tool_call.function.name, tool_call.function.arguments)
            log(f"[Tool] Result: {result}")

            tool_msg = {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": json.dumps(result),
            }
            messages.append(tool_msg)

        # Follow-up model call with tool outputs
        response = litellm.completion(
            model="bedrock/us.amazon.nova-pro-v1:0",
            messages=messages,
            tools=tool_config,
        )
        usage_followup = extract_tokens(response)
        for k in usage_total:
            usage_total[k] += usage_followup[k]

        final_message = response['choices'][0]['message']
        messages.append(final_message.__dict__)
    else:
        final_message = message

    return final_message.content, usage_total


In [ ]:
# Example usage
user_input = "What is the most popular song on WZPZ?"
converse_with_tools_litellm(user_input, bedrock_tools_to_litellm_tools(tool_config))